In [24]:
# interacting with Amazon AWS
import boto3
from sagemaker.session import Session

import os
import json
import numpy as np
import pandas as pd

In [6]:
structured_cik = [1146184, 1215680, 1224385, 1261467, 1591458, 1616344,  230611, 26617, 29648,  318336, 
                  356628, 42352, 58056, 68136, 72267,  782124,  803012,  851376,  853784,  867626,  87634,
                  877559,  890203,  895502, 91154,  920417,  922792]

## [National Information Center (NIC)](https://www.ffiec.gov/npw/FinancialReport/DataDownload)
**Tables of structure information for select banks and institutions for which the Federal Reserve has a supervisory, regulatory, or research interest:**
* Available for active institutions, the last instance of closed institutions, and branch offices
* Relationships data describes ownership between companies (i.e. parent and offspring relationship)
* Transformations data details mergers and acquisitions, includes those provided through government assistance

In [7]:
# read in both active and closed NIC data
rssd_closed = pd.read_csv('NIC/FFIEC_ATTRIBUTES_CLOSED.csv', low_memory=False)
rssd_active = pd.read_csv('NIC/FFIEC_ATTRIBUTES_ACTIVE.csv', low_memory=False)

# add column for the active status of a particular holding
rssd_active['STATUS'] = ['ACTIVE'] * rssd_active.shape[0]
rssd_closed['STATUS'] = ['CLOSED'] * rssd_closed.shape[0]

# combine both active and closed listing, extracting a subset of columns
all_rssd_id = pd.concat([rssd_active, rssd_closed])
all_rssd_id = all_rssd_id[['#ID_RSSD', 'NM_LGL', 'PRIM_FED_REG', 'CHTR_TYPE_CD', 
                           'ORG_TYPE_CD', 'ENTITY_TYPE', 'CITY', 'STATE_ABBR_NM', 
                           'CNTRY_NM', 'ID_TAX', 'ID_LEI']]

# re-map some terms to Federal Reserve Board identifiers
organization_map = {
    0 :'Not applicable (branches)',
    1: 'Corporation (stock)',
    2: 'General Partnership',
    3: 'Limited Partnership',
    4: 'Business Trust (fiduciary)',
    5: 'Sole Proprietorship',
    6: 'Mutual',
    9: 'Cooperative',
    10: 'LLP (Limited liability partnership)',
    11: 'LLC/C (Limited liability company/corporation)',
    12: 'Estate Trust',
    13: 'Limited Liability Limited Partnership',
    99: 'Other'}

charter_map = {
    0: 'Not available or not applicable (for branches)',
    110: 'Government Agency', 
    200: 'Commercial Bank',
    250: 'Non-deposit Trust Company',
    300: 'Savings Bank',
    310: 'Savings & Loan Association',
    320: 'Cooperative Bank',
    330: 'Credit Union (excluding Corporate Credit Union)',
    340: 'Industrial Bank',
    400: 'Edge or Agreement Corporation',
    500: 'Holding Company only',
    550: 'Insurance Broker or Agent',
    610: 'Employee Stock Ownership Plan/Trust (ESOP or ESOT)',
    700: 'Securities Broker and/or Dealer',
    710: 'Utility Company',
    720: 'Other Non-Depository Institution'}

entity_map = {
    'AGB': 'Agreement Corporation - Banking',
    'AGI': 'Agreement Corporation - Investment',
    'BHC': 'Bank Holding Company',
    'CPB': 'Cooperative Bank',
    'DBR': 'Domestic Branch of a Domestic Bank',
    'DEO': 'Domestic Entity Other',
    'DPS': 'Data Processing Servicer',
    'EBR': 'Edge Corporation - Domestic Branch',
    'EDB': 'Edge Corporation - Banking',
    'EDI': 'Edge Corporation - Investment',
    'FBH': 'Foreign Banking Organization as a BHC',
    'FBK': 'Foreign Bank',
    'FBO': 'Foreign Banking Organization',
    'FCU': 'Federal Credit Union',
    'FEO': 'Foreign Entity Other',
    'FHD': 'Financial Holding Company / BHC (Note: Can be a domestic or foreign-domiciled holding company)',
    'FHF': 'Financial Holding Company / FBO',
    'FNC': 'Finance Company',
    'FSB': 'Federal Savings Bank',
    'IBK': 'International Bank of a U.S. Depository - Edge or Trust Co.',
    'IBR': 'Foreign Branch of a U.S. Bank',
    'IHC': 'Intermediate Holding Company',
    'IFB': 'Insured Federal Branch of an FBO',
    'INB': 'International Non-bank Subs of Domestic Entities',
    'ISB': 'Insured State Branch of an FBO',
    'MTC': 'Non-deposit Trust Company - Member',
    'NAT': 'National Bank',
    'NMB': 'Non-member Bank',
    'NTC': 'Non-deposit Trust Company - Non-member',
    'NYI': 'New York Investment Company',
    'PST': 'Non-U.S. Branch managed by a U.S. Branch/Agency of a Foreign Bank for 002’s reporting - Pseudo Twig',
    'REP': 'Representative Office',
    'SAL': 'Savings & Loan Association',
    'SBD': 'Securities Broker / Dealer',
    'SCU': 'State Credit Union',
    'SLHC': 'Savings and Loan Holding Company',
    'SMB': 'State Member Bank',
    'SSB': 'State Savings Bank',
    'TWG': 'Non-U.S. Branch managed by a U.S. Branch/Agency of a Foreign Bank - TWIG',
    'UFA': 'Uninsured Federal Agency of an FBO',
    'UFB': 'Uninsured Federal Branch of an FBO',
    'USA': 'Uninsured State Agency of an FBO',
    'USB': 'Uninsured State Branch of an FBO'}

# remap according to dictionaries
all_rssd_id['CHTR_TYPE_CD'] = all_rssd_id['CHTR_TYPE_CD'].replace(charter_map)
all_rssd_id['ORG_TYPE_CD'] = all_rssd_id['ORG_TYPE_CD'].replace(organization_map)
all_rssd_id['ENTITY_TYPE'] = all_rssd_id['ENTITY_TYPE'].replace(entity_map)

all_rssd_id.to_csv('NIC/ALL-RSSD-ID.csv', index=False)

In [8]:
all_rssd_id.head()

,#ID_RSSD,NM_LGL,PRIM_FED_REG,CHTR_TYPE_CD,ORG_TYPE_CD,ENTITY_TYPE,CITY,STATE_ABBR_NM,CNTRY_NM,ID_TAX,ID_LEI
0,37,BANK OF HANCOCK COUNTY ...,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,SPARTA,GA,UNITED STATES,0,0
1,73,UTILITY EMPLOYEES FEDERAL CREDIT UNION ...,NCUA,Credit Union (excluding Corporate Credit Union),Mutual,Federal Credit Union,HOQUIAM,WA,UNITED STATES,910591861,0
2,242,FIRST COMMUNITY BANK XENIA-FLORA ...,FRS,Commercial Bank,Corporation (stock),State Member Bank,XENIA,IL,UNITED STATES,370274860,0
3,279,"MINEOLA COMMUNITY BANK, SSB ...",FDIC,Savings Bank,Mutual,State Savings Bank,MINEOLA,TX,UNITED STATES,750440734,54930067OMWEWTHVDO95
4,354,BISON STATE BANK ...,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,BISON,KS,UNITED STATES,0,549300T61O0X0RJMQ375


In [9]:
# create mappign convention for RSSDs and NAMES to be used in listing subordinates
rssd2name = dict(zip(all_rssd_id['#ID_RSSD'].values, all_rssd_id['NM_LGL'].values))

In [10]:
# extract the GVKEY for corresponding CIKs from the broker dealers
historical_names = pd.read_csv('wciklink_names.csv')
bd_names = historical_names[np.isin(historical_names.cik, structured_cik)]
bd_names = bd_names[['cik', 'fndate', 'n10k', 'coname', 'gvkey']]

In [11]:
bd_names.head()

,cik,fndate,n10k,coname,gvkey
1606,26617,2002-05-30,0.0,DAIWA SECURITIES AMERICA INC.,NaN
1607,26617,2010-05-28,0.0,DAIWA CAPITAL MARKETS AMERICA INC.,NaN
1811,29648,2003-03-03,0.0,PERSHING LLC,NaN
2720,42352,2003-01-28,0.0,"GOLDMAN, SACHS & CO.",NaN
2721,42352,2017-03-01,0.0,GOLDMAN SACHS & CO. LLC,NaN


In [12]:
# read in data from NIC pertaining to FFIEC mergers & aquistions
mergers_acquisitions = pd.read_csv('NIC/FFIEC_TRANSFORMATIONS.csv')

# rename transformation type to readable finacial interpretations (refer to FOMC Data Dictionary)
transformation_type = {1:'Charter Discontinued (Merger or Purchase & Assumption)', 5: 'Split', 7: 'Sale of Assets', 
                       9: 'Charter Retained (Merger or Purchase & Assumption)', 
                       50: 'Failure, Government Assistance Provided'}
mergers_acquisitions['TRNSFM_CD'] = mergers_acquisitions['TRNSFM_CD'].replace(transformation_type)

In [18]:
mergers_acquisitions.head()

,#ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,TRNSFM_CD,ACCT_METHOD,DT_TRANS
0,28,75026,01/01/1994 00:00:00,Charter Discontinued (Merger or Purchase & Ass...,0,19940101
1,46,1003848,10/01/1993 00:00:00,Charter Discontinued (Merger or Purchase & Ass...,0,19931001
2,55,951951,01/11/1982 00:00:00,Charter Discontinued (Merger or Purchase & Ass...,0,19820111
3,64,1213683,07/30/1988 00:00:00,"Failure, Government Assistance Provided",0,19880730
4,82,41393,01/01/2000 00:00:00,Charter Discontinued (Merger or Purchase & Ass...,0,20000101


In [16]:
# read in data from NIC pertaining to FFIEC mergers & aquistions
relationships = pd.read_csv('NIC/FFIEC_RELATIONSHIPS.csv')
relationships.head()

,#ID_RSSD_PARENT,ID_RSSD_OFFSPRING,D_DT_START,RELN_LVL,CTRL_IND,D_DT_RELN_EST,EQUITY_IND,OTHER_BASIS_IND,PCT_EQUITY,PCT_OTHER,...,REG_IND,PCT_EQUITY_BRACKET,PCT_EQUITY_FORMAT,D_DT_END,MB_COST,FC_IND,REGK_INV,DT_END,DT_RELN_EST,DT_START
0,130,1081305,06/27/1984 00:00:00,1,1,06/27/1984 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,02/14/1986 00:00:00,0,0,0,19860214,19840627,19840627
1,279,3923155,01/23/2008 00:00:00,1,1,01/23/2008 00:00:00,2,0,100.0,0.0,...,1,100,BRACKET,12/31/9999 00:00:00,0,0,0,99991231,20080123,20080123
2,505,2913168,05/27/1999 00:00:00,1,1,05/27/1999 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,06/29/2007 00:00:00,0,0,0,20070629,19990527,19990527
3,505,2913168,06/30/2007 00:00:00,1,1,05/27/1999 00:00:00,2,0,100.0,0.0,...,1,100,BRACKET,12/31/9999 00:00:00,0,0,0,99991231,19990527,20070630
4,505,3079566,06/01/2000 00:00:00,1,1,06/01/2000 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,02/12/2003 00:00:00,0,0,0,20030212,20000601,20000601


In [63]:
# read in data that maps RSSDs to CIKs
rssd2cik = pd.read_csv('NIC/RSSD-CIK.csv', low_memory=False)
rssd2cik['CIK'] = rssd2cik['CIK'].replace({769993:42352})
rssd2cik.head()

,#ID_RSSD,NM_LGL,PRIM_FED_REG,CHTR_TYPE_CD,ORG_TYPE_CD,ENTITY_TYPE,CITY,STATE_ABBR_NM,CNTRY_NM,ID_TAX,ID_LEI,CIK
0,43351,PEOPLES STATE BANK,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,MANHATTAN,KS,UNITED STATES,480308430,0,838010
1,44741,CITIZENS BANK,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,MOORESVILLE,IN,UNITED STATES,350227210,549300CABYW2DE2UTD57,230119
2,58243,JOHNSON BANK,FRS,Commercial Bank,Corporation (stock),State Member Bank,RACINE,WI,UNITED STATES,391141446,IWRZQFYIRJ0IMURZBB68,902143
3,58757,KENSINGTON BANK,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,KENSINGTON,MN,UNITED STATES,410257104,549300Q9WQZE336R0U94,1173376
4,58971,WASHINGTON TRUST BANK,FDIC,Commercial Bank,Corporation (stock),Non-member Bank,SPOKANE,WA,UNITED STATES,910462347,549300P2GEYNMH5OQA71,861787


**We want to examine the massive disparity that exists between the total RSSDs for both CLOSED and ACTIVE institutions, by considering whether firms have merged or been aquired as well as existing parent-child relationships. We first merge the FFIEC_TRANSFORMATIONS and follow with the FFIEC_RELATIONSHIPS data, selecting a few columns from each dataset.**

In [25]:
#  Amazon Textract client and Sagemaker session
s3 = boto3.client('s3')
session = Session()

bucket = 'ran-s3-systemic-risk'

# retrieving CIK-Dealers JSON file from s3 bucket
s3.download_file(bucket, 'Temp/CIKandDealers.json', 'temp.json')

# read all CIK and Dealer name information from storage
with open('temp.json', 'r') as f: cik2brokers = json.loads(f.read())

# remove local file after it has been created (variable is stored in memory)
os.remove('temp.json')

In [48]:
# the CIKS corresponding to a particular broker-dealer
broker_ciks = list(map(lambda x: int(x), cik2brokers['broker-dealers'].keys()))

In [28]:
# initialize all dataframes to be merged across RSSD-CIK
tb1 = rssd2cik
tb2 = mergers_acquisitions[['#ID_RSSD_PREDECESSOR','ID_RSSD_SUCCESSOR','TRNSFM_CD']]
tb3 = relationships[['#ID_RSSD_PARENT', 'ID_RSSD_OFFSPRING', 'RELN_LVL', 'REASON_ROW_CRTD']]

In [29]:
# merging the relationship data for RSSDs, highlighting parent and offspring relationships 
out_df1 = tb1.merge(tb3, left_on='#ID_RSSD', right_on='#ID_RSSD_PARENT')
out_df1['OFFSPRING_NM_LGL'] = out_df1['ID_RSSD_OFFSPRING'].replace(rssd2name)

In [30]:
out_df1.to_csv('NIC/RSSD-CIK-RELATIONSHIPS.csv', index=False)

In [31]:
# merge the M&A data with RSSD and CIK
out_df2 = tb1.merge(tb2, left_on='#ID_RSSD', right_on='ID_RSSD_SUCCESSOR')
out_df2['SUCCESSOR_NM_LGL'] = out_df2['ID_RSSD_SUCCESSOR'].replace(rssd2name)

In [32]:
out_df2.to_csv('NIC/RSSD-CIK-M&A.csv', index=False)

In [73]:
# extract a list of RSSD to CIK mappings restricting the sample to only broker-dealers
rssd2broker_cik = rssd2cik[np.isin(rssd2cik.CIK, broker_ciks)][['#ID_RSSD', 'CIK', 'NM_LGL']]
rssd2broker_cik.to_csv('NIC/broker-dealer-rssd-cik.csv', index=False)

In [80]:
rssd2broker_cik[np.isin(rssd2broker_cik.CIK, 
                        [782124, 42352, 68136, 91154, 72267, 1224385, 851376, 853784, 58056])]['#ID_RSSD'].values

array([1032389, 1146316, 1155420, 1440528, 1573239, 1900666, 2754521])